In [97]:
from mT5thai.mt5_thai_script import Mt5Thai
from mT5thai.webScrap import webScrap
import pandas as pd
import numpy as np
import html
from IPython.core.display import display, HTML
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import math

import warnings
warnings.simplefilter("ignore", UserWarning)

In [69]:
sentense = '''แนวโน้มของการใช้หุ่นยนต์ทางด้านการผลิตและงานบริการมีจำนวนสูงขึ้นมากกว่า 30% ทุกปี แต่ประเทศไทยยังขาดบุคลากรด้านวิทยาการหุ่นยนต์และระบบอัตโนมัติในทุกระดับ 
โดยเฉพาะอาชีพวิศวกร อาจารย์และนักวิจัย ฟีโบ้จึงได้รับการก่อตั้งขึ้น ในปี 2538 โดย รศ. ดร.ชิต เหล่าวัฒนา และฟีโบ้มีคติพจน์ว่า “A Cradle of Future Leaders in Robotics” 
ที่มุ่งสร้างผู้นำด้านวิทยาการหุ่นยนต์ของไทย
ฟีโบ้เป็นสถาบันระดับคณะแห่งแรกและแห่งเดียวในไทยที่เปิดสอนและให้ปริญญาครบทั้งระดับปริญญาตรี ปริญญาโท และปริญญาเอก ในสาขาวิทยาการหุ่นยนต์และระบบอัตโนมัติ ทุกหลักสูตรเน้นหลัก 
Outcome-based Education ที่บัณฑิตสามารถใช้สมรรถนะที่เรียนในการพัฒนาเทคโนโลยีหุ่นยนต์ขึ้นได้ ทำให้บัณฑิตฟีโบ้เป็นที่ต้องการของภาคเอกชนและภาครัฐอย่างมาก
นอกจากนี้ฟีโบ้มีผลงานวิจัย/พัฒนาและให้บริการวิชาการด้านหุ่นยนต์เพื่อการผลิตและเพื่อพัฒนาคุณภาพชีวิตมากกว่า 300 ระบบ โดยมีความร่วมมือที่แข็งแกร่งกับภาครัฐและภาคธุรกิจอุตสาหกรรมชั้นนำ 
ทั้งกลุ่มปิโตรเลียม อาหาร ฮาร์ดดิสก์ การผลิตไฟฟ้า การขนส่ง ธุรกิจบริการ การศึกษา การแพทย์และการฟื้นฟูสมรรถภาพ
ปัจจุบัน ฟีโบ้ โดยการนำของ รศ. ดร.สยาม เจริญเสียง ผู้อำนวยการสถาบันฯ มุ่งมั่นเพื่อยกระดับฟีโบ้เข้าสู่สถาบันชั้นนำระดับอาเซียนทางด้านวิทยาการหุ่นยนต์ 
รวมทั้งสร้างเครือข่ายพัฒนาหลักสูตรด้านหุ่นยนต์ ครูและนักเรียนของโรงเรียนทั่วประเทศมากกว่า 100 โรงเรียน'''

In [3]:
sentense2 = webScrap("https://www.thairath.co.th/news/local/northeast/2695085")

In [9]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, T5ForConditionalGeneration

model_name="thanathorn/mt5-cpe-kmutt-thai-sentence-sum"
# model_name = "Nopphakorn/mt5-news-thaisum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name, 
                        output_attentions=True,
                        output_scores=True,
                        # from_pt=True,
                        output_hidden_states=True,
                    )

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [127]:
input_ids = tokenizer.encode(sentense, return_tensors="pt", padding="longest")
min_length = 50
output = model.generate(
                        input_ids, 
                        min_length=min_length,     # Parameters that control the length of the output
                        max_length=min_length*2, 
                        do_sample=True,                 # Parameters that control the generation strategy used
                        num_beams=3,    
                        output_attentions=True,         # Parameters that define the output variables of `generate`
                        output_scores=True, 
                        return_dict_in_generate=True,
                        output_hidden_states=True,
                        )
output_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True)
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])


In [128]:
def html_escape(text):
    return html.escape(text)

def highlightedText(tokens, output, pad='', max_alpha=1, save_html=False) -> list:
    '''
    https://adataanalyst.com/machine-learning/highlight-text-using-weights/
    '''
    highlighted_text = []
    output_att = tf.stack(output.encoder_attentions)
    output_att.shape

    reshaped_scores = tf.reduce_mean(output_att[-1,:,0], axis=0)
    df = pd.DataFrame(reshaped_scores).sum(axis=0)
    df = 1/df
    for i, word in enumerate(tokens):
        
        if '▁' in word: word = word.replace('▁', ' ') # change space '▁' to ' '

        # df_sumcol = pd.DataFrame( attention[encoder1][0][encoder2] ).sum(axis=0) # reduce dim(N, N) to dim(N, 1) by sum column
        df /= df.max() # normalize

        weight = df[i]
        # print(weight)
        if weight < 0.5: weight = 0
        # else: weight = 0
        
        highlighted_text.append('<span style="background-color:rgba(251,155,59,' + str((weight) * max_alpha) + ');">' + html_escape(word) + '</span>')
        # highlighted_text.append('' + html_escape(word) + '')
        
    highlighted_text = pad.join(highlighted_text)

    display(HTML(highlighted_text))

    if save_html:
        html = HTML(highlighted_text).data
        # file name ex. attention_0_1.html
        with open(f'attention.html', 'w') as f:
            f.write(html)

    return highlighted_text

In [130]:
z = highlightedText(tokens, output)

In [ ]:
display(HTML(output_text))